In [67]:
import pickle

import numpy as np
import pandas as pd
import tomli
import torch
import xgboost as xgb
from Bio import SeqIO
from rdkit import Chem
from rdkit.Chem import AllChem
from sklearn.metrics import roc_auc_score

In [68]:
with open("src/config.toml", "rb") as f:
    config = tomli.load(f)

1. 数据读取(Data loading)

In [ ]:
# esp数据读取
# ESP data loading
df_test = pd.read_pickle(
    config["espData"]["encoded_path"] + "df_test_with_ESM1b_ts.pkl"
)
df_test = df_test.loc[df_test["ESM1b"] != ""]
df_test.reset_index(inplace=True, drop=True)

df_train = pd.read_pickle(
    config["espData"]["encoded_path"] + "df_train_with_ESM1b_ts.pkl"
)
df_train = df_train.loc[df_train["ESM1b"] != ""]
df_train.reset_index(inplace=True, drop=True)

/home/hanxd/miniconda3/envs/esp/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:82: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = libops.scalar_compare(x.ravel(), y, op)
/home/hanxd/miniconda3/envs/esp/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:82: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = libops.scalar_compare(x.ravel(), y, op)


In [ ]:
# Seq-P450数据读取
# Seq-P450 data loading
p450plant0 = pd.read_pickle(config["SeqP450Data"]["encoded_path"] + "p450plant0.pkl")
p450plant1 = pd.read_pickle(config["SeqP450Data"]["encoded_path"] + "p450plant1.pkl")
p450plant2 = pd.read_pickle(config["SeqP450Data"]["encoded_path"] + "p450plant2.pkl")
p450plant3 = pd.read_pickle(config["SeqP450Data"]["encoded_path"] + "p450plant3.pkl")
p450plant4 = pd.read_pickle(config["SeqP450Data"]["encoded_path"] + "p450plant4.pkl")

p450plant = pd.concat(
    [p450plant0, p450plant1, p450plant2, p450plant3, p450plant4], ignore_index=True
)

2. 物种数据过滤与提取(Filtering and extracting species data)

In [ ]:
# 选择物种
# Select species
deletedataname = "Arabidopsis_thaliana"
deletedataname = "Erigeron_breviscapus"
deletedataname = "Glycine_max"
deletedataname = "Zea_mays"

In [ ]:
# 读取pair
# Load pairs
deletedata = pd.read_csv(
    config["SeqP450Data"]["pair_path"] + "P450_Substrate.txt",
    sep="\t",
    header=None,
    names=["enzyme", "substrate", "sequence"],
)
deletedata = deletedata.sample(frac=1, random_state=42)

# 读取序列
# Load sequences
fasta_file = config["screeningData"]["enzyme_path"] + f"{deletedataname}.fasta"
sequences = SeqIO.to_dict(SeqIO.parse(fasta_file, "fasta"))

# 放入dataframe
# Insert into DataFrame
for index, row in deletedata.iterrows():
    nowenzyme = row["enzyme"]
    nowsubstrate = row["substrate"]
    try:
        target_sequence = sequences[nowenzyme].seq
    except:
        target_sequence = ""
    deletedata.loc[index, "sequence"] = str(target_sequence)

In [ ]:
# 异常值检测
# Outlier detection
deletedata = deletedata[deletedata["sequence"] != ""]
has_null_values = deletedata.isna().any().any()
has_empty_strings = deletedata.applymap(lambda x: x == "")
if has_empty_strings.any().any():
    print("DataFrame contains empty strings.")
if has_null_values:
    print("DataFrame contains empty values.")

In [ ]:
# 使用esm进行序列特征提取（仅需要执行一次即可）
# Sequence feature extraction using ESM (only needs to be run once)

# ! python src/codes/extract.py esm1b_t33_650M_UR50S {config['screeningData']['enzyme_path']}{deletedataname}.fasta {config['screeningData']['esm_path']} --repr_layers 33 --include mean

Transferred model to GPU
Read data/screeningData/enzyme/Zea_mays.fasta with 18 sequences
Processing 1 of 3 batches (7 sequences)
Processing 2 of 3 batches (7 sequences)
Processing 3 of 3 batches (4 sequences)


In [ ]:
# 读取特征
# Load features
deletedata["ESM1b"] = ""
deletedata["ECFP"] = ""


for ind in deletedata.index:
    esms = torch.load(
        config["screeningData"]["esm_path"] + str(deletedata["enzyme"][ind]) + ".pt"
    )
    sdf_file_path = (
        config["SeqP450Data"]["substrate_path"] + deletedata["substrate"][ind] + ".sdf"
    )
    mol = Chem.MolFromMolFile(sdf_file_path)
    ecfpso = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=1024).ToBitString()
    deletedata["ESM1b"][ind] = esms["mean_representations"][33].tolist()
    deletedata["ECFP"][ind] = ecfpso
# 已存在的反应，设置标签为1
# Assign label 1 to reactions that already exist
deletedata["Binding"] = 1

In [ ]:
# 异常值检测
# Outlier detection
has_null_values = deletedata.isna().any().any()
has_empty_strings = deletedata.applymap(lambda x: x == "")
if has_empty_strings.any().any():
    print("DataFrame contains empty strings.")
if has_null_values:
    print("DataFrame contains empty values.")

In [ ]:
# 已知反应的数据存储
# Data storage for known reactions
deletedata.to_pickle(
    config["screeningData"]["encoded_path"] + f"{deletedataname}_deletedata.pkl"
)

In [78]:
deletedata

,enzyme,substrate,sequence,ESM1b,ECFP,Binding
101,CYP706C37,MCAR,MEATAPANLFYAAFLFLSVLYLAVTRRRTSRLPPGPTGLPLVGSLP...,"[-0.031913720071315765, 0.11623724550008774, 0...",0000101000000000000000000000000101001000000000...,1
79,CYP81A39,ZEA,METAYVVVLSFFFIVAIHRLLNRGESKNMSKQPLPPGPRAIPVLGH...,"[0.00890614464879036, 0.05576604604721069, 0.0...",0000000000000000000000000000000001001000000000...,1
33,CYP79A61,LTR,MVSSPQANKFPLRKQISSPSSSCMAPHVLLVLVILLYLVRTLRPWR...,"[-0.0023888919968158007, 0.14067767560482025, ...",0100000000000000000000000000000000000000000000...,1
22,CYP701A43,kenA1,MQSLLAGIPAGGAAAAAAVGGLVAAAALAERADQKNRLNLPPAVPG...,"[-0.03364456444978714, 0.13790029287338257, 0....",0000100000000000000000000000000001001000000000...,1
368,CYP711A3Zm,CAA,MGRQPLVIVANAELCKEVGIKKFKSMPNRSLPSAIANSPIHLKGLF...,"[-0.07262156158685684, 0.13780206441879272, 0....",0000101000000000000000000000000101001000000000...,1
220,CYP71C4,IND,MAAQLHHALYELLHEAAAAQRALLLAIPFSLLLLPLLLRYLAASAS...,"[-0.06542030721902847, 0.09601475298404694, 0....",0000000000000000000000000000000000000000000000...,1
250,CYP71Z19,BIS,MEQKVLVAVGVAVLLVVVLSKLKSVLVTKPKLNLPPGPWTLPLIGS...,"[-0.07729123532772064, 0.1101357489824295, 0.0...",0000000000000000000000000000000001000000000000...,1
23,CYP701A26,KEN,MESLVAALPAGGAAAAAAFGGLVAAAALAGKVGLVGSKKHLNAPPA...,"[-0.02506798319518566, 0.1476520299911499, -0....",0000100000000000000000000000000001011000000000...,1
218,CYP71C3,HBOA,MALQAAYEYLQQAVGHGAWSSTQTLTLLLIAVPTVLLLLASLAKST...,"[-0.047974780201911926, 0.11363550275564194, 0...",0000000100100000000000000000000000001000000000...,1
3,CYP78A1,LAC,MAMASAACSCTDGTWWVYALPALLGSDTLCAHPALLAGLIFLATVS...,"[0.03729096055030823, 0.03447350487112999, 0.1...",0000000000000000000000000000000001000000000000...,1


In [ ]:
# 从Seq-P450的数据中删除该物种的待测试正样本数据
# Remove the positive test samples of the species from Seq-P450 data
filtered_rows = p450plant[
    (p450plant["Binding"] == 1) & p450plant["enzyme"].isin(deletedata["enzyme"])
]
p450plant_filtered = p450plant.drop(filtered_rows.index)
p450plant_filtered.reset_index(drop=True, inplace=True)

3. 特征提取与模型训练(Feature extraction and model training)

In [80]:
def create_input_and_output_data(df):
    X = ()
    y = ()
    for ind in df.index:
        emb = df["ESM1b"][ind]
        ecfp = np.array(list(df["ECFP"][ind])).astype(int)

        X = X + (np.concatenate([ecfp, emb]),)
        y = y + (df["Binding"][ind],)

    return (X, y)


train_X, train_y = create_input_and_output_data(df=df_train)
test_X, test_y = create_input_and_output_data(df=df_test)
train_new_X, train_new_y = create_input_and_output_data(df=p450plant_filtered)

train_X = np.concatenate([train_X, train_new_X])
train_y = np.concatenate([train_y, train_new_y])

feature_names = ["ECFP_" + str(i) for i in range(1024)]
feature_names = feature_names + ["ESM1b_" + str(i) for i in range(1280)]

In [81]:
param = {
    "learning_rate": 0.31553117247348733,
    "max_delta_step": 1.7726044219753656,
    "max_depth": 10,
    "min_child_weight": 1.3845040588450772,
    "num_rounds": 342.68325188584106,
    "reg_alpha": 0.531395259755843,
    "reg_lambda": 3.744980563764689,
    "weight": 0.26187490421514203,
}


num_round = param["num_rounds"]
param["objective"] = "binary:logistic"

weights = np.array(
    [param["weight"] if binding == 0 else 1.0 for binding in np.array(train_y)]
)

del param["num_rounds"]
del param["weight"]

dtrain = xgb.DMatrix(
    np.array(train_X),
    weight=weights,
    label=np.array(train_y),
    feature_names=feature_names,
)
dtest = xgb.DMatrix(
    np.array(test_X), label=np.array(test_y), feature_names=feature_names
)
dtrain_p450 = xgb.DMatrix(
    np.array(train_new_X), label=np.array(train_new_y), feature_names=feature_names
)

evallist = [(dtrain, "train"), (dtrain_p450, "dtrain_p450")]
bst = xgb.train(param, dtrain, int(num_round), evallist, verbose_eval=10)
y_test_pred = np.round(bst.predict(dtest))
acc_test = np.mean(y_test_pred == np.array(test_y))
roc_auc = roc_auc_score(np.array(test_y), bst.predict(dtest))

print("Accuracy on test set: %s, ROC-AUC score for test set: %s" % (acc_test, roc_auc))

[0]	train-error:0.26947	dtrain_p450-error:0.56034
[10]	train-error:0.14555	dtrain_p450-error:0.40313
[20]	train-error:0.09312	dtrain_p450-error:0.24984
[30]	train-error:0.06435	dtrain_p450-error:0.19896
[40]	train-error:0.04892	dtrain_p450-error:0.15721
[50]	train-error:0.03626	dtrain_p450-error:0.12003
[60]	train-error:0.02900	dtrain_p450-error:0.09589
[70]	train-error:0.02237	dtrain_p450-error:0.07175
[80]	train-error:0.01878	dtrain_p450-error:0.05219
[90]	train-error:0.01538	dtrain_p450-error:0.04110
[100]	train-error:0.01265	dtrain_p450-error:0.03196
[110]	train-error:0.01090	dtrain_p450-error:0.02479
[120]	train-error:0.00880	dtrain_p450-error:0.02153
[130]	train-error:0.00764	dtrain_p450-error:0.01761
[140]	train-error:0.00661	dtrain_p450-error:0.01696
[150]	train-error:0.00572	dtrain_p450-error:0.01370
[160]	train-error:0.00515	dtrain_p450-error:0.01305
[170]	train-error:0.00464	dtrain_p450-error:0.01044
[180]	train-error:0.00418	dtrain_p450-error:0.01044
[190]	train-error:0.003

In [ ]:
# 模型保存
# Model saving
pickle.dump(
    bst,
    open(
        config["screeningData"]["model_path"] + f"{deletedataname}_deletedatamodel.dat",
        "wb",
    ),
)